## Chapter 4: Hands-On usage of infospace, xdata event, infospace call-back, timer,web callback

For the Application to be configurable, we inherit from xdata::ActionListener, attach to and implement the callback "actionPerformed"for the xdata Event "xdaq-event:setDefaultValues". 

All applications loaded by the executive share a special infospace whose handle we acquire with getApplicationInfoSpace(). We declare a named parameter in the application infospace by firing itemavailable event. The framework listens to this event and matches the application properties in the xml file with parameters in application infospaces and fires "xdaq-event:setDefaultValues" when done. xmlns= attribute in the application property refers to the name of the application infospace.

   <properties xmlns="urn:xdaq-application:bril::timesource::Application" xsi:type="soapenc:Struct">
      <AAA xsi:type="xsd:boolean">true</AAA>
   </properties>

Our application get notified only if it is a infospace event listener, so we must add appInfoSpace->addListener(this, "urn:xdaq-event:setDefaultValues");

While Configuration is a usecase of sharing data using the special application infospace, Monitoring is a more general usecase of infospace infrastructure. We can create as many infospaces as we need. Different applications in the same process can attach to a common infospace to coordinate activities or exchange data. For example, Application A and B are attached to a common infospace, A will do something only if variable "run" is changed by B. 

Here's a table of frequently used events:

event | what happened | callback | how to listen (is=infospace) | how to fire (is=infospace)
   --- | --- | --- | ---
   urn:xdaq-event:setDefaultValues | application properties acquired | actionPerformed(xdata::Event) | is->addListener(this,"urn:xdaq-event:setDefaultValues") | not you
   eventing::api::BusReadyToPublish | the bus is ready to publish | actionPerformed(toolbox::Event) | getEventingBus(name).addActionListener | not you
   urn:xdata-event:ItemAvailableEvent | a parameter is registered | actionPerformed(xdata::Event) | is->addItemAvailableListener | is->fireItemAvailable
   urn:xdata-event:ItemGroupChangedEvent | a group of parameters changed | actionPerformed(xdata::Event) | is->addGroupChangedListener | is->fireItemGroupChanged
   urn:xdata-event:ItemChangedEvent | a parameter changed | actionPerformed(xdata::Event) | is->addItemChangedListener | is->fireItemValueChanged
   urn:xdata-event:ItemGroupRetrieveEvent | update a group of parameters | implicit | is->addGroupRetrieveListener | is->fireItemGroupRetrieve
   urn:xdata-event:ItemRetrieveEvent | update a parameter | implicit | is->addItemRetrieveListener | is->fireItemValueRetrieve

### Example 1
A xdaq application who controls a queue and monitors its status via hyperdaq. 

Export again from svn example directory and set up your working environment. 

Choose either modify the configuration file by hand $workdir/daq/bril/mypackage/xml/MonitorWebPull.xml 

*or* 

generate it from template:

```
cd $workdir/daq/bril/scripts

./generatexml.sh mypackage/MonitorWebPull

```

Build mypackage library:


```
make install

```

Go to where the configuration xml resides and launch the executive from there:

```
/opt/xdaq/bin/xdaq.exe -p $your_context_port -e /opt/xdaq/etc/default.profile -c `pwd`/MonitorWebPull.xml
```

Open the hyperdaq page of the application.

The "Pop" button triggers the action of popping the queue by one (see printout on screen).

The "Pull" button is a request to update the monitored current size of the queue on the web.

Such on-demand monitoring of a variable is called the "Pull" mode in the xdaq manuals.

Observe in the source code : 

How to create an infospace and declared named parameters in it?

Where the queue is initialized and why?

Where the queue is popped and why?

Where the "Pull" monitored variables happened? 

### Example 2

We split the above program into two where the datasource application
pops the queue on timer while an external monitoring collector
application get notified as soon as the status of the queue is updated.

Repeat the same setup procedure as Example 1 for configuration MonitoringCollector.

```
/opt/xdaq/bin/xdaq.exe -p $your_context_port -e /opt/xdaq/etc/default.profile -c `pwd`/MonitoringCollector/.xml

```

This context contains two applications. DataSource sets up a queue and pop the element on a timer where the queue and timer setup are configurable from the configuration xml. Upon popping a queue element, DataSource fires a infospace group changed event. MonitoringCollector is a listener to the group changed event of all infospaces with name pattern as configured. Upon notified by its event subscription, it logs the current status of the variables being observed.  

Such on-notification monitoring of a variable is called the "Push" mode in the xdaq manuals.

*Note*: 

Our MonitoringCollector class is an emulation of the monitoring collecting mechanism available via XAAS (XDAQ As A Service) in the BRIL zone. You do not have to write anything like this by yourself. It is for you to be aware that by firing ItemGroupChanged event, you are potentially sending a notification an external observer.

How you organize the monitoring variables into groups can be externalized as a "flashlist" or a "flashlist" is a monitoring matrix definition. From inside the application, we just know about monitorable groups.

### Exercise


Compare with the HelloWorld example, we inherited from more base classes. What are the purpose of each of them and which are the corresponding callback methods?

In Example 2, run two instances of DataSource in the same context to see what happens
(Uncomment the block in the configuration xml). Observe how "id=" and "instance=" fields are configured.

